# Homework 2.2: The Quest For A Better Network

In this assignment you will build a monster network to solve Tiny ImageNet image classification.

This notebook is intended as a sequel to seminar 3, please give it a try if you haven't done so yet.

(please read it at least diagonally)

* The ultimate quest is to create a network that has as high __accuracy__ as you can push it.
* There is a __mini-report__ at the end that you will have to fill in. We recommend reading it first and filling it while you iterate.
 
## Grading
* starting at zero points
* +20% for describing your iteration path in a report below.
* +20% for building a network that gets above 20% accuracy
* +10% for beating each of these milestones on __TEST__ dataset:
    * 25% (50% points)
    * 30% (60% points)
    * 32.5% (70% points)
    * 35% (80% points)
    * 37.5% (90% points)
    * 40% (full points)
    
## Restrictions
* Please do NOT use pre-trained networks for this assignment until you reach 40%.
 * In other words, base milestones must be beaten without pre-trained nets (and such net must be present in the anytask atttachments). After that, you can use whatever you want.
* you __can't__ do anything with validation data apart from running the evaluation procedure. Please, split train images on train and validation parts

## Tips on what can be done:


 * __Network size__
   * MOAR neurons, 
   * MOAR layers, ([torch.nn docs](http://pytorch.org/docs/master/nn.html))

   * Nonlinearities in the hidden layers
     * tanh, relu, leaky relu, etc
   * Larger networks may take more epochs to train, so don't discard your net just because it could didn't beat the baseline in 5 epochs.

   * Ph'nglui mglw'nafh Cthulhu R'lyeh wgah'nagl fhtagn!


### The main rule of prototyping: one change at a time
   * By now you probably have several ideas on what to change. By all means, try them out! But there's a catch: __never test several new things at once__.


### Optimization
   * Training for 100 epochs regardless of anything is probably a bad idea.
   * Some networks converge over 5 epochs, others - over 500.
   * Way to go: stop when validation score is 10 iterations past maximum
   * You should certainly use adaptive optimizers
     * rmsprop, nesterov_momentum, adam, adagrad and so on.
     * Converge faster and sometimes reach better optima
     * It might make sense to tweak learning rate/momentum, other learning parameters, batch size and number of epochs
   * __BatchNormalization__ (nn.BatchNorm2d) for the win!
     * Sometimes more batch normalization is better.
   * __Regularize__ to prevent overfitting
     * Add some L2 weight norm to the loss function, PyTorch will do the rest
       * Can be done manually or like [this](https://discuss.pytorch.org/t/simple-l2-regularization/139/2).
     * Dropout (`nn.Dropout`) - to prevent overfitting
       * Don't overdo it. Check if it actually makes your network better
   
### Convolution architectures
   * This task __can__ be solved by a sequence of convolutions and poolings with batch_norm and ReLU seasoning, but you shouldn't necessarily stop there.
   * [Inception family](https://hacktilldawn.com/2016/09/25/inception-modules-explained-and-implemented/), [ResNet family](https://towardsdatascience.com/an-overview-of-resnet-and-its-variants-5281e2f56035?gi=9018057983ca), [Densely-connected convolutions (exotic)](https://arxiv.org/abs/1608.06993), [Capsule networks (exotic)](https://arxiv.org/abs/1710.09829)
   * Please do try a few simple architectures before you go for resnet-152.
   * Warning! Training convolutional networks can take long without GPU. That's okay.
     * If you are CPU-only, we still recomment that you try a simple convolutional architecture
     * a perfect option is if you can set it up to run at nighttime and check it up at the morning.
     * Make reasonable layer size estimates. A 128-neuron first convolution is likely an overkill.
     * __To reduce computation__ time by a factor in exchange for some accuracy drop, try using __stride__ parameter. A stride=2 convolution should take roughly 1/4 of the default (stride=1) one.
 
   
### Data augmemntation
   * getting 5x as large dataset for free is a great 
     * Zoom-in+slice = move
     * Rotate+zoom(to remove black stripes)
     * Add Noize (gaussian or bernoulli)
   * Simple way to do that (if you have PIL/Image): 
     * ```from scipy.misc import imrotate,imresize```
     * and a few slicing
     * Other cool libraries: cv2, skimake, PIL/Pillow
   * A more advanced way is to use torchvision transforms:
   
    ```
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    trainset = torchvision.datasets.ImageFolder(root=path_to_tiny_imagenet, train=True, download=True, transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
    ```
   
   * Or use this tool from Keras (requires theano/tensorflow): [tutorial](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html), [docs](https://keras.io/preprocessing/image/)
   * [Albumentations](https://github.com/albumentations-team/albumentations) is another awesome solution.
   * Stay realistic. There's usually no point in flipping dogs upside down as that is not the way you usually see them.  
    * But sometimes there is! Some examples of advanced image augmentation approaches: [mixup](https://arxiv.org/pdf/1710.09412.pdf), [cutmix](https://arxiv.org/pdf/1905.04899.pdf)   

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision, torchvision.transforms as transforms

# Uncomment this to disable "Skipping walk through <class 'list'>" warnings in DataSphere's env
# %enable_full_walk

In [2]:
!wget https://raw.githubusercontent.com/borisshapa/Practical_DL/spring21/homework02/tiny_img.py

--2022-05-02 21:53:29--  https://raw.githubusercontent.com/borisshapa/Practical_DL/spring21/homework02/tiny_img.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4422 (4.3K) [text/plain]
Saving to: ‘tiny_img.py.3’

tiny_img.py.3       100%[===================>]   4.32K  --.-KB/s    in 0s      

2022-05-02 21:53:29 (73.8 MB/s) - ‘tiny_img.py.3’ saved [4422/4422]



In [3]:
# downloading TinyImagenet
# you don't have to run this cell more than once

from tiny_img import download_tinyImg200, fix_test_data
data_path = '.'
download_tinyImg200(data_path)
fix_test_data(data_path)

./tiny-imagenet-200.zip


mkdir: cannot create directory ‘./tiny-imagenet-200/val/n01443537’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n01443537/images’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n01629819’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n01629819/images’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n01641577’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n01641577/images’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n01644900’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n01644900/images’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n01698640’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n01698640/images’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n01742172’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n01742172/images’: File exist

mkdir: cannot create directory ‘./tiny-imagenet-200/val/n02814533’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n02814533/images’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n02814860’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n02814860/images’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n02815834’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n02815834/images’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n02823428’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n02823428/images’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n02837789’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n02837789/images’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n02841315’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n02841315/images’: File exist

mkdir: cannot create directory ‘./tiny-imagenet-200/val/n04070727/images’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n04074963’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n04074963/images’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n04099969’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n04099969/images’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n04118538’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n04118538/images’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n04133789’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n04133789/images’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n04146614’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n04146614/images’: File exists
mkdir: cannot create directory ‘./tiny-imagenet-200/val/n04149813’: File exist

We will split `tiny-imagenet-200/train` dataset into train and val parts, and use  `tiny-imagenet-200/val` dataset as a test one.

You are free to use either the default ImageFolder Dataset, or the custom one, which will read and store the whole data in RAM. The second one is preferable only when you have a slow disk; make sure then you do have an extra couple of GiBs of memory (it also could take some time to load the images):

In [4]:
import os

means = np.random.rand(3)
std = np.random.rand(3)

transform_augment = transforms.Compose([
                                        transforms.RandomRotation([-15, 15]),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor(),
                                        transforms.Normalize(means, std)                                       
])

imagenet_dir = os.path.join(data_path, 'tiny-imagenet-200')
dataset = torchvision.datasets.ImageFolder(imagenet_dir + '/train', transform=transforms.ToTensor())
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [80000, 20000],
                                                           generator=torch.Generator().manual_seed(42))
train_dataset.dataset.transform = transform_augment
test_dataset = torchvision.datasets.ImageFolder(imagenet_dir + '/val', transform=transforms.ToTensor())

# OR

# from tiny_img_ram import TinyImagenetRAM
# dataset = TinyImagenetRAM('tiny-imagenet-200/train', transform=transforms.ToTensor())
# train_dataset, val_dataset = torch.utils.data.random_split(dataset, [80000, 20000],
#                                                            generator=torch.Generator().manual_seed(42))
# test_dataset = TinyImagenetRAM('tiny-imagenet-200/val', transform=transforms.ToTensor())

In [5]:
# feel free to copypaste code from seminar03 as a basic template for training

In [6]:
model = BorisNet(3, 64).cuda()

from typing import Union

class BorisNet(nn.Module):
    def __init__(self, in_channels: int, out_channels: int):
        super().__init__()
        self.model = nn.Sequential(
            BorisNet.__get_2conv_pool_block(in_channels, out_channels, kernel_size = 3, pooling = 4),
            BorisNet.__get_2conv_pool_block(out_channels, out_channels * 2, kernel_size = 3, pooling = 4),
            BorisNet.__get_2conv_pool_block(out_channels * 2, out_channels * 4, kernel_size = 3, pooling = 2),
            BorisNet.__get_2conv_pool_block(out_channels * 4, out_channels * 8, kernel_size = 3, pooling = 2),
            nn.Flatten(),
            nn.Linear(out_channels * 8, 200),
        )
    
    @staticmethod
    def __get_conv_bn_relu_dropout_block(in_channels: int, out_channels: int, kernel_size: int = 3, padding: Union[str, int] = "same"):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
    
    @staticmethod
    def __get_2conv_pool_block(in_channels: int, out_channels: int, kernel_size: int = 3, padding: Union[str, int] = "same", stride: int = 1, pooling: int = 2):
        return nn.Sequential(
            BorisNet.__get_conv_bn_relu_dropout_block(in_channels, out_channels, kernel_size, padding),
            BorisNet.__get_conv_bn_relu_dropout_block(out_channels, out_channels, kernel_size, padding),
            nn.AvgPool2d(kernel_size=pooling)
        )

    def forward(self, input):
        return self.model(input)

In [20]:
model = BorisNet(3, 64).cuda()

In [21]:
from torch.optim import Adam
from tqdm import tqdm
from matplotlib import pyplot as plt
from IPython.display import clear_output
from sklearn.metrics import accuracy_score
from IPython.display import clear_output

criterion = nn.CrossEntropyLoss()
opt = Adam(model.parameters(), lr=0.001)
epochs = 100
batch_size = 4096
log_every = 10

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

batch_ind = 0

accuracy = []
train_loss = []

for epoch in range(1, epochs + 1):
    for batch in tqdm(train_dataloader):
        batch_ind += 1

        images, labels = batch

        model.zero_grad()

        images = images.cuda()
        labels = labels.cuda()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        opt.step()

        train_loss.append(loss.item())

        if batch_ind % log_every == 0:

            preds = []
            true_labels = []
    
            model.eval()
            with torch.no_grad():
                for batch in val_dataloader:
                    images, labels = batch
                    images = images.cuda()
                    labels = labels.cuda()
                    
                    output = model(images)
                    pred_labels = torch.argmax(output, dim=1)

                    preds.append(pred_labels.cpu().detach())
                    true_labels.append(labels.cpu().detach())
            
            preds = torch.cat(preds).numpy()
            true_labels = torch.cat(true_labels).numpy()

            accuracy.append(accuracy_score(true_labels, preds))
            model.train()
    
        clear_output(wait=True)

        fig = plt.figure(figsize=(15, 6))
        ax1 = fig.add_subplot(1, 2, 1)
        ax2 = fig.add_subplot(1, 2, 2)
        ax1.plot(train_loss)
        ax2.plot(accuracy)
        plt.show()

  0%|                            | 0/20 [00:01<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 4.00 GiB (GPU 0; 39.41 GiB total capacity; 33.51 GiB already allocated; 206.50 MiB free; 37.28 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [17]:
test_dataloader = DataLoader(test_dataset, batch_size=16)

preds = []
true_labels = []

model.eval()

for batch in test_dataloader:
    images, labels = batch
    images = images.cuda()
    labels = labels.cuda()

    output = model(images)
    pred_labels = torch.argmax(output, dim=1)

    preds.append(pred_labels.cpu().detach())
    true_labels.append(labels.cpu().detach())
    
preds = torch.cat(preds).numpy()
true_labels = torch.cat(true_labels).numpy()

test_acc = accuracy_score(true_labels, preds)

When everything is done, please calculate accuracy on `tiny-imagenet-200/val`

In [18]:
test_accuracy = test_acc # YOUR CODE

In [19]:
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_accuracy * 100))

if test_accuracy * 100 > 40:
    print("Achievement unlocked: 110lvl Warlock!")
elif test_accuracy * 100 > 35:
    print("Achievement unlocked: 80lvl Warlock!")
elif test_accuracy * 100 > 30:
    print("Achievement unlocked: 70lvl Warlock!")
elif test_accuracy * 100 > 25:
    print("Achievement unlocked: 60lvl Warlock!")
else:
    print("We need more magic! Follow instructons below")

Final results:
  test accuracy:		1.08 %
We need more magic! Follow instructons below


```

```

```

```

```

```


# Report

All creative approaches are highly welcome, but at the very least it would be great to mention
* the idea;
* brief history of tweaks and improvements;
* what is the final architecture and why?
* what is the training method and, again, why?
* Any regularizations and other techniques applied and their effects;


There is no need to write strict mathematical proofs (unless you want to).
 * "I tried this, this and this, and the second one turned out to be better. And i just didn't like the name of that one" - OK, but can be better
 * "I have analized these and these articles|sources|blog posts, tried that and that to adapt them to my problem and the conclusions are such and such" - the ideal one
 * "I took that code that demo without understanding it, but i'll never confess that and instead i'll make up some pseudoscientific explaination" - __not_ok__

### Hi, my name is `___ ___`, and here's my story

A long time ago in a galaxy far far away, when it was still more than an hour before the deadline, i got an idea:

##### I gonna build a neural network, that
* brief text on what was
* the original idea
* and why it was so

How could i be so naive?!

##### One day, with no signs of warning,
This thing has finally converged and
* Some explaination about what were the results,
* what worked and what didn't
* most importantly - what next steps were taken, if any
* and what were their respective outcomes

##### Finally, after __  iterations, __ mugs of [tea/coffee]
* what was the final architecture
* as well as training method and tricks

That, having wasted ____ [minutes, hours or days] of my life training, got

* accuracy on training: __
* accuracy on validation: __
* accuracy on test: __


[an optional afterword and mortal curses on assignment authors]